In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("ggplot")

import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms  
from torch.utils.data.sampler import SubsetRandomSampler  
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import torch.nn.functional as F
import os

from tqdm import tqdm
import tqdm.notebook as tq

import time
import json
import copy
import gc
import os
import time
import random
from datetime import datetime

from PIL import Image
from tqdm.notebook import tqdm
from sklearn import model_selection, metrics

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(1001)

In [ ]:
TEST_PATH = '/kaggle/input/cassava-leaf-disease-classification/test_images/'
MODEL_PATH = (
    "../input/deit-facebook/DEIY-facebook-fin848.pt"
)

# model specific global variables
IMG_SIZE = 224

In [ ]:
class CassavaDatasettest(torch.utils.data.Dataset):
    """
    Helper Class to create the pytorch dataset
    """

    def __init__(self, df, data_path=TEST_PATH, mode="test", transforms=None):
        super().__init__()
        self.df_data = df.values
        self.data_path = data_path
        self.transforms = transforms
        self.mode = mode
        self.data_dir = "train_images" if mode == "train" else "test_images"

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        img_name  , _ = self.df_data[index]
        #print(img_name)
        img_path = os.path.join(self.data_path, img_name)
        img = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            image = self.transforms(img)

        return image

In [ ]:
BATCH_SIZE = 32


In [ ]:
transforms_test = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)


In [ ]:
dftest = pd.read_csv(os.path.join('/kaggle/input/cassava-leaf-disease-classification/', "sample_submission.csv")) 

In [ ]:
test_ds = CassavaDatasettest(dftest,data_path=TEST_PATH, transforms=transforms_test )


In [ ]:
dftest.head()

In [ ]:
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, pin_memory=False) 


In [ ]:
# Load
trained_model = torch.load(MODEL_PATH)
trained_model.eval()

In [ ]:

gpu = torch.cuda.is_available()

device = torch.device("cuda" if gpu else "cpu")   # use CPU or GPU



In [ ]:

df = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')

trained_model.to(device)
trained_model.eval()

preds_torch = []
with torch.no_grad():
    test_tqdm = valid_bar = tq.tqdm(test_loader, total=len(test_loader), desc="Testing", position=0, leave=True)
    for images in test_tqdm:
        images = images.to(device)
        preds_torch.extend(trained_model(images).detach().cpu().numpy()) 

In [ ]:
torch_outcomes = pd.concat([dftest['image_id'], pd.DataFrame(preds_torch)], axis=1).sort_values(['image_id'])


torch_outcomes

In [ ]:
final_preds = (torch_outcomes.drop('image_id', axis=1)).to_numpy().argmax(1)

final_preds

In [ ]:
final_preds[0]

In [ ]:
submit = pd.DataFrame({'image_id': torch_outcomes['image_id'].values, 'label': final_preds})
submit.to_csv('submission.csv', index=False)

        
           








In [ ]:
submit.head()